# LLM 에 도구 바인딩(Binding Tools)

LLM 모델이 도구(tool) 를 호출할 수 있으려면 chat 요청을 할 때 모델에 도구 스키마(tool schema) 를 전달해야 합니다. 

도구 호출(tool calling) 기능을 지원하는 LangChain Chat Model 은 `.bind_tools()` 메서드를 구현하여 LangChain 도구 객체, Pydantic 클래스 또는 JSON 스키마 목록을 수신하고 공급자별 예상 형식으로 채팅 모델에 바인딩(binding) 합니다.

바인딩된 Chat Model 의 후속 호출은 모델 API에 대한 모든 호출에 도구 스키마를 포함합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("02-Bind-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
02-Bind-Tools


## LLM에 바인딩할 Tool 정의

실험을 위한 도구(tool) 를 정의합니다.

- `get_word_length` : 단어의 길이를 반환하는 함수
- `add_function` : 두 숫자를 더하는 함수
- `naver_news_crawl` : 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

**참고**
- 도구를 정의할 때 `@tool` 데코레이터를 사용하여 도구를 정의합니다.
- docstring 은 가급적 영어로 작성하는 것을 권장합니다.

In [3]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구를 정의합니다.
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]

## bind_tools() 로 LLM 에 도구 바인딩

llm 모델에 `bind_tools()` 를 사용하여 도구를 바인딩합니다.

In [5]:
from langchain_openai import AzureChatOpenAI

# 모델 생성
llm = AzureChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

실행결과를 확인합니다. 

결과는 `tool_calls` 에 저장됩니다. 따라서, `.tool_calls` 를 확인하여 도구 호출 결과를 확인할 수 있습니다.

**참고**
- `name` 은 도구의 이름을 의미합니다.
- `args` 는 도구에 전달되는 인자를 의미합니다.

In [22]:
# 실행 결과
llm_with_tools.invoke("What is the length of the word '-------------------------'?").tool_calls

[{'name': 'get_word_length',
  'args': {'word': '------------------------'},
  'id': 'call_gxHYDAS4E9fIzuX08TwKq6iK',
  'type': 'tool_call'}]

다음으로는 `llm_with_tools` 와 `JsonOutputToolsParser` 를 연결하여 `tool_calls` 를 parsing 하여 결과를 확인합니다.

In [11]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word '-------------------------'?")

In [12]:
print(tool_call_results)

[{'args': {'word': '------------------------'}, 'type': 'get_word_length'}]


실행 결과는 다음과 같습니다.

**참고**
- `type`: 도구의 이름
- `args`: 도구에 전달되는 인자

In [23]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': '------------------------'}, 'type': 'get_word_length'}]


get_word_length
{'word': '------------------------'}


도구 이름과 일치하는 도구를 찾아 실행합니다.

In [24]:
tool_call_results[0]["type"], tools[0].name

('get_word_length', 'get_word_length')

`execute_tool_calls` 함수는 도구를 찾아 args 를 전달하여 도구를 실행합니다.

즉, `type` 은 도구의 이름을 의미하고 `args` 는 도구에 전달되는 인자를 의미합니다.

In [25]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행합니다.
execute_tool_calls(tool_call_results)

[실행도구] get_word_length [Argument] {'word': '------------------------'}
[실행결과] 24


## bind_tools + Parser + Execution

이번에는 일련의 과정을 한 번에 실행합니다.

- `llm_with_tools` : 도구를 바인딩한 모델
- `JsonOutputToolsParser` : 도구 호출 결과를 파싱하는 파서
- `execute_tool_calls` : 도구 호출 결과를 실행하는 함수

**흐름 정리**
1. 모델에 도구를 바인딩
2. 도구 호출 결과를 파싱
3. 도구 호출 결과를 실행

In [26]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [27]:
# 실행 결과
chain.invoke("What is the length of the word '-------------------------'?")

[실행도구] get_word_length [Argument] {'word': '------------------------'}
[실행결과] 24


In [28]:
# 실행 결과
chain.invoke("114.5 + 121.2")
print(114.5 + 121.2)

[실행도구] add_function [Argument] {'a': 114.5, 'b': 121.2}
[실행결과] 235.7
235.7


In [29]:
# 실행 결과
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘:https://n.news.naver.com/mnews/article/648/0000032735"
)

[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/article/648/0000032735'}
[실행결과] 삼성바이오, CMO·시밀러 쌍끌이에 '4조 클럽' 가입

연결매출 4.5조·영업익 1.3조 '역대 최대'올해 매출 20~25% 성장 기대…5조 돌파
삼성바이오로직스가 지난해 20% 이상의 고성장을 이어가며 사상 최대인 4조5000억원의 매출을 기록했다. 바이오의약품 위탁생산(CMO) 및 바이오시밀러 사업 모두에서 역대 최대 실적을 낸 결과다. 삼성바이오로직스는 국내 바이오제약 최초로 '매출 4조 클럽'에 이름을 올리게 됐다.22일 금융감독원에 따르면 삼성바이오로직스는 지난해 연결 기준 매출 4조5473억원, 영업이익 1조3201억원을 기록했다. 전년대비 매출액은 23%(8527억원), 영업이익은 19%(2064억원) 각각 증가했다.삼성바이오로직스, 4조 클럽…올해도 20%이상 성장먼저 별도기준으로 삼성바이오로직스는 4공장 매출 상승 및 1~3공장 풀가동을 바탕으로 매출 3조4971억원, 영업이익 1조3214억원을 기록했다. 전년 대비 매출은 19%(5583억원), 영업이익은 10%(1172억원) 증가한 것으로 나타났다. 삼성바이오로직스는 지난해 글로벌 제약사와 잇단 위탁생산 계약을 체결하며 역대급 수주 성과를 보였다. 지난해 수주액만 43억달러로 2011년 창사 이래 누적 수주 총액은 176억 달러를 돌파했다. 현재 글로벌 상위 20곳 제약사 중 총 17곳을 고객사로 확보하고 있다.삼성바이오로직스는 올해 하반기 4공장이 풀가동되고 5공장(4월 준공 예정)까지 상업 생산에 돌입하면 매출 확대에 큰 기여를 할 것으로 기대하고 있다. 삼성바이오로직스는 2027년 가동을 목표로 6공장 건설도 계획하고 있다. 삼성바이오에피스, 영업이익 2300억 증가 '사상 최대'글로벌 시장에 바이오시밀러를 공급하는 삼성바이오에피스는 매출 1조5377억원, 영업이익 4354억원을 각각 기

## bind_tools > Agent & AgentExecutor 로 대체

`bind_tools()` 는 모델에 사용할 수 있는 스키마(도구)를 제공합니다. 

`AgentExecutor` 는 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성합니다.

**참고**
- `Agent` 와 `AgentExecutor` 에 대해서는 다음 장에서 자세히 다룹니다.

In [46]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(k=6)

In [47]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = AzureChatOpenAI(model="gpt-4o-mini", temperature=0)

In [48]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length, add_function, naver_news_crawl, search]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [40]:
# Agent 실행
result = agent_executor.invoke({"input": "How many letters in the word `-------------------------'?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': '-------------------------'}`


25The word `-------------------------'` has 25 letters.

> Finished chain.
The word `-------------------------'` has 25 letters.


In [41]:
# Agent 실행
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7114.5 + 121.2의 계산 결과는 235.7입니다.

> Finished chain.
114.5 + 121.2의 계산 결과는 235.7입니다.


한 번의 실행으로 끝나는 것이 아닌, 모델이 자신의 결과를 확인하고 다시 자신을 호출하는 과정을 거칩니다.

In [34]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

# 결과 확인
print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3
Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


이번에는 뉴스 결과를 크롤링 해서 요약 해달라는 요청을 수행합니다.


In [35]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사 내용을 크롤링해줘:https://n.news.naver.com/mnews/article/648/0000032735"
    }
)
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/648/0000032735'}`


삼성바이오, CMO·시밀러 쌍끌이에 '4조 클럽' 가입

연결매출 4.5조·영업익 1.3조 '역대 최대'올해 매출 20~25% 성장 기대…5조 돌파
삼성바이오로직스가 지난해 20% 이상의 고성장을 이어가며 사상 최대인 4조5000억원의 매출을 기록했다. 바이오의약품 위탁생산(CMO) 및 바이오시밀러 사업 모두에서 역대 최대 실적을 낸 결과다. 삼성바이오로직스는 국내 바이오제약 최초로 '매출 4조 클럽'에 이름을 올리게 됐다.22일 금융감독원에 따르면 삼성바이오로직스는 지난해 연결 기준 매출 4조5473억원, 영업이익 1조3201억원을 기록했다. 전년대비 매출액은 23%(8527억원), 영업이익은 19%(2064억원) 각각 증가했다.삼성바이오로직스, 4조 클럽…올해도 20%이상 성장먼저 별도기준으로 삼성바이오로직스는 4공장 매출 상승 및 1~3공장 풀가동을 바탕으로 매출 3조4971억원, 영업이익 1조3214억원을 기록했다. 전년 대비 매출은 19%(5583억원), 영업이익은 10%(1172억원) 증가한 것으로 나타났다. 삼성바이오로직스는 지난해 글로벌 제약사와 잇단 위탁생산 계약을 체결하며 역대급 수주 성과를 보였다. 지난해 수주액만 43억달러로 2011년 창사 이래 누적 수주 총액은 176억 달러를 돌파했다. 현재 글로벌 상위 20곳 제약사 중 총 17곳을 고객사로 확보하고 있다.삼성바이오로직스는 올해 하반기 4공장이 풀가동되고 5공장(4월 준공 예정)까지 상업 생산에 돌입하면 매출 확대에 큰 기여를 할 것으로 기대하고 있다. 삼성바이오로직스는 2027년 가동을 목표로 6공장 건설도 계획하고 있다. 삼성바이오에피스, 영업이익 2300억 증가 '사상 최대'글로벌 시장에 바이오시밀러를 공급하는 삼성바

In [50]:
result = agent_executor.invoke(
    {"input": "2025년 현재 미국 대통령은 누구인가?"}
)
print(result["output"])





> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2025년 미국 대통령'}`


[{'url': 'https://www.youtube.com/watch?v=3rQ40CsQgwU', 'content': '미국의 47대 대통령으로 도널드 트럼프 당선인이 취임합니다. 8년 전 취임식에서 "미국을 다시 위대하게!"를 외쳤던 트럼프 당선인이 4년 만에 다시'}, {'url': 'https://ko.wikipedia.org/wiki/2025년_도널드_트럼프_대통령_취임식', 'content': '2025년 도널드 트럼프 대통렴의 취임식 장면. 2025년 도널드 트럼프 대통령 취임식은 도널드 트럼프의 제47대 미국의 대통령 취임식이다. 2025년 1월 20일, 워싱턴 d.c.에 있는 미국 국회의사당 로턴다에서 개최되었다. 이는 미국 대통령의 60번째 취임식이자 도널드 트럼프의 두 번째 대통령 취임식다.'}, {'url': 'https://www.klog.kr/news/트럼프-취임일-공약-11가지', 'content': '2025년 1월 20일 제47대 미국 대통령 도널드 트럼프의 취임일은 2025년 1월 20일로 예정되어 있습니다(현지 시간). 많은 이들이 그의 취임 첫날에 주목하는 이유는, 그가 선거 유세 중 "취임 첫날만큼은 독재자가 되고 싶다"고 언급하며 강력한 정책 추진을 예고했기'}, {'url': 'https://www.economidaily.com/view/20250121021542316', 'content': '도널드 트럼프가 2025년 1월 20일 월요일 워싱턴 미 국회의사당 원형홀에서 열린 제60대 대통령 취임식에서 멜라니아 트럼프가 성경을 들고 있는 가운데 존 로버츠 대법원장으로부터 미국의 제47대 대통령으로 취임했다.[사진=로이터 연합뉴스]'}, {'url': 'https://www.bbc.com/korean/articles/c